In [37]:
#  get files
import pandas as pd
df = pd.read_csv(r"C:\Users\christine\Desktop\datamining\selected_scam_images.csv") # CHANGE HERE
img_folder = r"C:\Users\christine\Desktop\datamining\data\scamimage" + "\x5c"  #CHANGE HERE

# x = df.values.tolist()
# x.to_csv('real_images.csv')
df.shape

df = df.iloc[0:500]

(6612, 1)

In [41]:
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.applications.xception import Xception
from keras.models import load_model
from pickle import load
import numpy as np
from PIL import Image
from glob import glob
import json

def extract_features(filename, model):
        try:
            image = Image.open(filename)
            
        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

def word_for_id(integer, tokenizer):
 for word, index in tokenizer.word_index.items():
     if index == integer:
         return word
 return None


def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

max_length = 32 
tokenizer = load(open("tokenizer.p","rb"))
model = load_model('models/model_9.h5')
xception_model = Xception(include_top=False, pooling="avg")
labels = {}

problematic = []

for i in df.images:
    try: 
        path = (img_folder +i)
        photo = extract_features(path, xception_model)
        img = Image.open(path)
        description = generate_desc(model, tokenizer, photo, max_length)
        description = description[6:-4]
        labels[i] = description
        print(path)

    except:
        problematic.append(i)
    

json_object = json.dumps(labels) 

# change the filename to your name
with open('scam_0_500.json', 'w') as f:
  json.dump(labels, f, ensure_ascii=False)

print('\n\n DONE ALR :DDD')

1/1 [==============================] - 0s 168ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\keneth1.jpg
1/1 [==============================] - 0s 157ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\Jhjh281.jpg
1/1 [==============================] - 0s 159ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\OlegWilliams1.jpg
1/1 [==============================] - 0s 163ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\brucep1.jpg
1/1 [==============================] - 0s 159ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\brucep2.jpg
1/1 [==============================] - 0s 169ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\brucep3.jpg
1/1 [==============================] - 0s 192ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\brucep4.jpg
1/1 [==============================] - 0s 193ms/step
C:\Users\christine\Desktop\datamining\data\scamimage\brucep5.jpg
1/1 [==============================] - 0s 180ms/st